In [2]:
## Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os
import imblearn
from imblearn.pipeline import make_pipeline as make_pipeline_imbfinal 
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn import svm
from sklearn.metrics import *
import zipfile
from zipfile import ZipFile
from io import BytesIO
import sys
import glob
import datetime
import time
import boto.s3
from boto.s3.key import Key 
import pickle
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df = pd.read_csv('liver1.csv')

In [31]:
col_list=['age','tot_bilirubin','direct_bilirubin','tot_proteins','albumin','ag_ratio','sgpt'
          ,'alkphos'
          #,'sgot','gender_Male']
         ]
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train[col_list]
y_train=df_train['is_patient']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test[col_list]
y_test=df_test['is_patient']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)
print(x_train.shape,y_train.shape)

(338, 8) (338,)


In [32]:
accuracy =[]
model_name =[]
dataset=[]
f1score = []
precision = []
recall = []
true_positive =[]
false_positive =[]
true_negative =[]
false_negative =[]

In [33]:
logreg = LogisticRegression()
## fitiing the model
logreg.fit(x_train_sc, y_train)
filename = 'logReg_model.sav'
pickle.dump(logreg,open(filename,'wb'))
logreg

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
prediction = logreg.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
dataset.append('Training')
model_name.append('Logistic Regression')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[ 42,  55],
       [  6, 235]], dtype=int64)

In [35]:
feature_list = list(x_train.columns)
importances = list(logreg.coef_)
print(importances)
print(x_train.columns)

[array([-0.47300518, -0.24585048, -0.71008588, -0.16558173, -1.07602462,
       -1.43043243,  0.26202595, -0.03683913])]
Index(['age', 'tot_bilirubin', 'direct_bilirubin', 'tot_proteins', 'albumin',
       'ag_ratio', 'sgpt', 'alkphos'],
      dtype='object')


In [36]:
rfc = RandomForestClassifier(n_estimators=50,random_state=42)
## fitiing the model
rfc.fit(x_train_sc, y_train)
filename = 'RFC_model.sav'
pickle.dump(rfc,open(filename,'wb'))
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [37]:
prediction = rfc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Random Forest Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[ 95,   2],
       [  0, 241]], dtype=int64)

In [38]:
feature_list = list(x_train.columns)
importances = list(rfc.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: ag_ratio             Importance: 0.18
Variable: direct_bilirubin     Importance: 0.14
Variable: albumin              Importance: 0.14
Variable: tot_proteins         Importance: 0.13
Variable: age                  Importance: 0.12
Variable: tot_bilirubin        Importance: 0.11
Variable: sgpt                 Importance: 0.09
Variable: alkphos              Importance: 0.08


In [30]:
col_list=['age','tot_bilirubin','direct_bilirubin','tot_proteins','albumin','ag_ratio','sgpt'
          ,'alkphos'
          #,'sgot','gender_Male']
         ]
df_train,df_test = train_test_split(df,train_size=0.7,random_state=42)
x_train=df_train[col_list]
y_train=df_train['is_patient']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=df_test[col_list]
y_test=df_test['is_patient']
scaler.fit(x_test)
x_test_sc=scaler.transform(x_test)
print(x_train.shape,y_train.shape)

ValueError: labels ['alkphos'] not contained in axis

In [12]:
rfc = RandomForestClassifier(n_estimators=50,random_state=0)
## fitiing the model
rfc.fit(x_train_sc, y_train)
rfc

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
prediction = rfc.predict(x_train_sc)
f1 = f1_score(y_train, prediction)
p = precision_score(y_train, prediction)
r = recall_score(y_train, prediction)
a = accuracy_score(y_train, prediction)
cm = confusion_matrix(y_train, prediction)
tp = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tn = cm[1][1]
model_name.append('Random Forest Classifier')
dataset.append('Training')
f1score.append(f1)
precision.append(p)
recall.append(r)
accuracy.append(a)
true_positive.append(tp) 
false_positive.append(fp)
true_negative.append(tn) 
false_negative.append(fn)
cm

array([[288,   0],
       [  1, 119]], dtype=int64)

In [14]:
feature_list = list(x_train.columns)
importances = list(rfc.feature_importances_)
feature_importances = [(x_train, round(importance, 2)) for x_train, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: albumin              Importance: 0.16
Variable: tot_proteins         Importance: 0.15
Variable: ag_ratio             Importance: 0.15
Variable: age                  Importance: 0.12
Variable: tot_bilirubin        Importance: 0.09
Variable: sgpt                 Importance: 0.09
Variable: sgot                 Importance: 0.09
Variable: direct_bilirubin     Importance: 0.08
Variable: alkphos              Importance: 0.07


Conclusion : We have found after feature selection that these 11 columns have more importance and it's better to make the model with these.